In [2]:
import numpy as np
import cv2
import os
import random
import skimage
from matplotlib import pyplot as plt

from PIL import Image


In [3]:
def rgb_to_lms():
        """
        Matrix for RGB color-space to LMS color-space transformation.
        """
        return np.array([[17.8824, 43.5161, 4.11935],
                         [3.45565, 27.1554, 3.86714],
                         [0.0299566, 0.184309, 1.46709]]).T

def lms_to_rgb() -> np.ndarray:
    """
    Matrix for LMS colorspace to RGB colorspace transformation.
    """
    return np.array([[0.0809, -0.1305, 0.1167],
                     [-0.0102, 0.0540, -0.1136],
                     [-0.0004, -0.0041, 0.6935]]).T

def lms_protanopia_sim(degree: float = 1.0) -> np.ndarray:
    """
    Matrix for Simulating Protanopia colorblindness from LMS color-space.
    :param degree: Protanopia degree.
    """
    return np.array([[1 - degree, 2.02344 * degree, -2.52581 * degree],
                     [0, 1, 0],
                     [0, 0, 1]]).T

def lms_deutranopia_sim(degree: float = 1.0) -> np.ndarray:
    """
    Matrix for Simulating Deutranopia colorblindness from LMS color-space.
    :param degree: Deutranopia degree.
    """
    return np.array([[1, 0, 0],
                     [0.494207 * degree, 1 - degree, 1.24827 * degree],
                     [0, 0, 1]]).T


def load_lms(path):
    """
    Image loader.
    :param path: File path (absolute)
    """
    img_rgb = np.array(Image.open(path)) / 255
    img_lms = np.dot(img_rgb[:,:,:3], rgb_to_lms())

    return img_lms

In [8]:
for category in ["Tlight", "Chimney", "Bicycle", "Other", "Motorcycle", "Car", "Bus"]
    for i in range(1, 21):
        
        # Colorblind
        lms = load_lms(os.path.join(os.getcwd(),f"CAPTCHA/{category} ({i}).png"))
        
        img_sim = np.dot(lms, lms_protanopia_sim())
        img_sim = np.uint8(np.dot(img_sim, lms_to_rgb()) * 255)
        
        cv2.imwrite(os.path.join(os.getcwd(),f"CAPTCHA/colorblind/{category}_protanopia ({i}).png"), img_sim)
        
        img_sim = np.dot(lms, lms_deutranopia_sim())
        img_sim = np.uint8(np.dot(img_sim, lms_to_rgb()) * 255)
        
        cv2.imwrite(os.path.join(os.getcwd(),f"CAPTCHA/colorblind/{category}_tritanopia ({i}).png"), img_sim)
        
        
        # Grayscale
        img = cv2.imread(os.path.join(os.getcwd(),f"CAPTCHA/{category} ({i}).png"))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        cv2.imwrite(os.path.join(os.getcwd(),f"CAPTCHA/colorblind/{category}_grayscale ({i}).png"), gray)
        
        
        # Blurry
        blur = cv2.GaussianBlur(img,(25,25),0) #bigger tuple =blurrier
        cv2.imwrite(os.path.join(os.getcwd(),f"CAPTCHA/blur/{category}_gaussian ({i}).png"), blur)
        
        # Diabetic Retinopathy
        height, width = img.shape[:2]

        seed = random.randint(0, 492)
        rng = np.random.default_rng(seed)

        noise = rng.integers(0, 255, (height,width), np.uint8, True)
        
        # sigma controls number
        blur = cv2.GaussianBlur(noise, (0,0), sigmaX=9, sigmaY=9, borderType = cv2.BORDER_DEFAULT)

        stretch = skimage.exposure.rescale_intensity(blur, in_range='image', out_range=(0,255)).astype(np.uint8)

        # lower threshold controls spread
        thresh = cv2.threshold(stretch, 162, 255, cv2.THRESH_BINARY)[1]
        
        # split into shapes
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8,8))
        mask = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.merge([mask,mask,mask])

        # add mask to input
        output = cv2.add(img, mask)
        
        # change white to black
        white = np.where(
            (output[:, :, 0] == 255) & 
            (output[:, :, 1] == 255) & 
            (output[:, :, 2] == 255)
        )

        output[white] = [0, 0, 0]
        
        cv2.imwrite(os.path.join(os.getcwd(),f"CAPTCHA/diabetes/{category}_diabetes ({i}).png"), gray)
        

True